In [248]:
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.applications.densenet import DenseNet121
from tensorflow.python.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop

In [249]:
train_dir = "D:/moinul_data/Data Pre Processing 3/Train"
val_dir = "D:/moinul_data/Data Pre Processing 3/Validation"
test_dir = "D:/moinul_data/Data Pre Processing 3/Test"

In [250]:
model = DenseNet121(include_top=False,input_shape=(64, 64, 3), weights='imagenet')

In [251]:
def print_layer_trainable():
    for layer in model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

True:	input_4
True:	zero_padding2d_6
True:	conv1/conv
True:	conv1/bn
True:	conv1/relu
True:	zero_padding2d_7
True:	pool1
True:	conv2_block1_0_bn
True:	conv2_block1_0_relu
True:	conv2_block1_1_conv
True:	conv2_block1_1_bn
True:	conv2_block1_1_relu
True:	conv2_block1_2_conv
True:	conv2_block1_concat
True:	conv2_block2_0_bn
True:	conv2_block2_0_relu
True:	conv2_block2_1_conv
True:	conv2_block2_1_bn
True:	conv2_block2_1_relu
True:	conv2_block2_2_conv
True:	conv2_block2_concat
True:	conv2_block3_0_bn
True:	conv2_block3_0_relu
True:	conv2_block3_1_conv
True:	conv2_block3_1_bn
True:	conv2_block3_1_relu
True:	conv2_block3_2_conv
True:	conv2_block3_concat
True:	conv2_block4_0_bn
True:	conv2_block4_0_relu
True:	conv2_block4_1_conv
True:	conv2_block4_1_bn
True:	conv2_block4_1_relu
True:	conv2_block4_2_conv
True:	conv2_block4_concat
True:	conv2_block5_0_bn
True:	conv2_block5_0_relu
True:	conv2_block5_1_conv
True:	conv2_block5_1_bn
True:	conv2_block5_1_relu
True:	conv2_block5_2_conv
True:	conv2_blo

In [252]:
datagen_train = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
                                    width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=False)
datagen_val = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)

In [253]:
batch_size = 30
input_shape = (64,64)
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_val = datagen_test.flow_from_directory(directory=val_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

generator_test = datagen_finaltest.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)


Found 4468 images belonging to 2 classes.
Found 639 images belonging to 2 classes.
Found 655 images belonging to 2 classes.


In [254]:
steps_val = generator_val.n / batch_size

steps_test = generator_test.n / batch_size

In [255]:
transfer_layer = model.get_layer('relu')
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)

In [256]:
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.

new_model.add(Dense(512, activation='relu'))


# Add the final layer for the actual classification.
new_model.add(Dense(2, activation='softmax'))

In [257]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']
new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [258]:
epochs = 10
steps_per_epoch = 100
history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_val,
                                  validation_steps=steps_val)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 21.3 steps
Epoch 1/10
100/100 [==============================] - 17s 170ms/step - loss: 0.3993 - categorical_accuracy: 0.8476 - val_loss: 1.0369 - val_categorical_accuracy: 0.4288
Epoch 2/10
100/100 [==============================] - 5s 53ms/step - loss: 0.2681 - categorical_accuracy: 0.9016 - val_loss: 0.7491 - val_categorical_accuracy: 0.6479
Epoch 3/10
100/100 [==============================] - 5s 53ms/step - loss: 0.2081 - categorical_accuracy: 0.9280 - val_loss: 0.6785 - val_categorical_accuracy: 0.7465
Epoch 4/10
100/100 [==============================] - 5s 54ms/step - loss: 0.1825 - categorical_accuracy: 0.9333 - val_loss: 0.6267 - val_categorical_accuracy: 0.8013
Epoch 5/10
100/100 [==============================] - 5s 53ms/step - loss: 0.1470 - categorical_accuracy: 0.9463 - val_loss: 0.5241 - val_categorical_accuracy: 0.8357
Epoch 6/10
100/100 [==============================] - 5s 53ms/step -

In [212]:
#new_model.save("D:/resnetMRI.h5")

In [260]:
import cv2
import os
from tqdm import tqdm

In [273]:
path = "D:/moinul_data/Data Pre Processing 3/Test/No"

test_data = np.empty((1, 64, 64, 3))

for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
    try:
        img_array = cv2.imread(os.path.join(path,img))  # convert to array
        new_array = cv2.resize(img_array, (64, 64))  # resize to normalize data size
        
        #new_array = new_array.reshape((64, 64))
        
       
        new_array = new_array.reshape((-1, 64, 64, 3))
        #print(new_array.shape)
        test_data = np.append(test_data, new_array, axis = 0)  # add this to our training_data
    except Exception as e:  # in the interest in keeping the output clean...
        print(e)
        pass
        #except OSError as e:
        #    print("OSErrroBad img most likely", e, os.path.join(path,img))
        #except Exception as e:
        #    print("general exception", e, os.path.join(path,img))



100%|██████████| 426/426 [00:03<00:00, 120.14it/s]


In [274]:
test_data.shape

(427, 64, 64, 3)

In [275]:
scaled_data = test_data/255
new_model.predict_classes(scaled_data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,